In [ ]:
import pandas as pd
import plotly.graph_objects as go # pip install plotly
import numpy as np
from plotly.graph_objs import *

In [ ]:
ECG_Features = pd.read_csv("DREAMER_ECG.csv")
# Number of features : Nf = 30
Nf=ECG_Features.shape[1]-1
ECG_Features.head()

In [ ]:
ECG_Feature_Value=[]
for k in range(0,23):
    for j in range(0,18):
        for f in range(0,Nf):
            if not(np.isnan(ECG_Features.iloc[18*k+j,1+f]) or np.isinf(ECG_Features.iloc[18*k+j,1+f])):
                ECG_Feature_Value.append(ECG_Features.iloc[18*k+j,1+f])
            else:
                ECG_Feature_Value.append(0)

In [ ]:
m=np.min(ECG_Feature_Value)
M=np.max(ECG_Feature_Value)
Colors=[]
Color_Amplification=10
for c in range(0,len(ECG_Feature_Value)):
    Colors.append('rgb('+str(round(255*Color_Amplification*(ECG_Feature_Value[c]-m)/(M-m),1))+','+str(round(255*(ECG_Feature_Value[c]-m)/(M-m),1))+','+str(round(255*(ECG_Feature_Value[c]-m)/(M-m),1))+')')
#print(Colors)

In [ ]:
print([np.tile(np.tile(c,23),18)for c in range(0,Nf)])

In [ ]:
fig = go.Figure(data=go.Scatter3d(
    x = np.tile([c for c in range(0,23)],18*Nf)
    ,y = np.tile([c for c in range(0,18)],23*Nf)
    ,z = np.tile([c for c in range(0,Nf)],18*23)
    #text = planets,
    ,mode = 'markers',
    marker = dict(
        sizemode = 'diameter',
        size = 5, #ECG_Feature_Value,
        color = Colors#['rgb(0,0,255)' for c in range(0,18*23*Nf)],
        )
))
fig.update_layout(
    title="Bubble Plot for the ECG features",
    scene=Scene(
        xaxis=XAxis(title='Subject number'),
        yaxis=YAxis(title='Video number'),
        zaxis=ZAxis(title='Feature number')
    )
    #zaxis_title="Feature number",
    #font=dict(
    #    family="Courier New, monospace",
    #    size=18,
    #    color="#7f7f7f")
    )
fig.show()                    

In [ ]:
from ipywidgets import widgets
import plotly.express as px

ECG_Feature_Widget = widgets.Dropdown(
    options=list(ECG_Features.columns[1:Nf]),
    value='ECG_Rate_Mean',
    description='ECG Feature :',
)

#graphic = go.Histogram(x=ECG_Features['ECG_Rate_Mean'], opacity=0.75, name='Some data')
data=ECG_Features['ECG_Rate_Mean']
graphic = go.Histogram(x=data, opacity=0.75, name='Some data')
g = go.FigureWidget(data=[graphic],
                    layout=go.Layout(
                        title=dict(
                            text='Interactive visualization of the ECG Features (Histogram)'
                        ),
                        barmode='overlay',
                        xaxis=dict(title = ''), yaxis=dict(title = 'Counts[#]')
                    ))

def validate():
    if ECG_Feature_Widget.value in ECG_Features[1:Nf].unique():
        return True
    else:
        return False


def response(change):
            g.data[0].x = ECG_Features[ECG_Feature_Widget.value]
            g.layout.barmode = 'overlay'
            g.layout.xaxis.title = ''
            g.layout.yaxis.title = 'Counts[#]'


ECG_Feature_Widget.observe(response, names="value")

container = widgets.HBox([ECG_Feature_Widget])
widgets.VBox([container,g])